In [1]:
!pip install numpy torch torchvision opencv-python mediapipe

  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached cffi-1.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 8.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import torch
import torchvision.io as io
import torchvision.transforms as T
import cv2
import mediapipe as mp

# Constants
IMG_SIZE = 216
FRAMES = 20
CHANNELS = 3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CLASSES = ['Non-suspicious', 'Suspicious']

print(f"Using device: {DEVICE}")

# # Initialize MediaPipe FaceMesh
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True  # Enable iris landmarks (468–477)
)


# Selected 71 Landmark Indices
selected_indices = [
    # Jawline
    93, 132, 58, 172, 136, 150, 149, 176, 152,
    377, 400, 378, 379, 365, 397, 288, 361, 323,

    # Left Eye
    33, 160, 158, 133, 153, 144, 145, 468,

    # Right Eye
    362, 385, 387, 263, 373, 380, 381, 473,

    # Mouth (upper + lower lips)
    61, 39, 37, 0, 267, 269, 291, 375, 405, 314, 17, 84, 181, 146,

    # Nose (bridge + sides)
    6, 197, 195, 5, 4, 1, 19, 94, 48, 279,

    # Eyebrows
    63, 105, 66, 107, 336, 296, 334, 293,

    # Forehead (light coverage)
    54, 67, 10, 297, 284
]

# Resize
resize_transform = T.Resize((IMG_SIZE, IMG_SIZE))

# Frame and Landmark Extraction
def extract_video_frames_landmarks_motion(video_path, num_frames, img_size, visualize=False):
    try:
        video, _, _ = io.read_video(video_path, pts_unit='sec')
    except Exception as e:
        print(f"Error reading {video_path}: {e}")
        return None, None, None, None

    total_frames = video.shape[0]
    if total_frames < 1:
        return None, None, None, None

    indices = torch.linspace(0, total_frames - 1, num_frames).long()
    frames = video[indices]  # (T, H, W, C)
    frames = frames.permute(0, 3, 1, 2).float() / 255.0
    frames = frames.cpu()

    resized_frames = []
    landmarks_all = []

    for frame in frames:
        frame_np = frame.permute(1, 2, 0).numpy()
        frame_bgr = (frame_np * 255).astype(np.uint8)
        frame_bgr = cv2.cvtColor(frame_bgr, cv2.COLOR_RGB2BGR)

        results = face_mesh.process(frame_bgr)

        frame_landmarks = []
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            for idx in selected_indices:
                if idx < len(face_landmarks.landmark):
                    lm = face_landmarks.landmark[idx]
                    frame_landmarks.append([lm.x, lm.y])
                else:
                    frame_landmarks.append([0, 0])
        else:
            frame_landmarks = [[0, 0]] * len(selected_indices)

        frame_tensor = torch.tensor(frame_np).permute(2, 0, 1)
        resized = resize_transform(frame_tensor)
        resized = resized.permute(1, 2, 0).numpy()

        resized_frames.append(resized)
        landmarks_all.append(frame_landmarks)

    resized_frames = np.array(resized_frames)  # (T, 216, 216, 3)
    landmarks_all = np.array(landmarks_all)    # (T, 71, 2)

    # Compute Motion (delta)
    motion = landmarks_all[1:] - landmarks_all[:-1]  # (T-1, 71, 2)
    motion = np.vstack([np.zeros((1, 71, 2)), motion])  # pad first frame with zeros

    # Compute Velocity (motion normalized by time gap)
    time_gap = 0.75  # implied 0.75 seconds between frames
    velocity = motion / time_gap

    # Visualization (optional)
    if visualize:
        frame_idx = 0
        frame_vis = (resized_frames[frame_idx] * 255).astype(np.uint8)
        h, w = frame_vis.shape[:2]
        frame_landmarks = landmarks_all[frame_idx]
        frame_landmarks_scaled = (frame_landmarks * np.array([w, h])).astype(np.int32)

        for (x, y) in frame_landmarks_scaled:
            if x > 0 and y > 0:
                cv2.circle(frame_vis, (x, y), 2, (0, 255, 0), -1)

        save_dir = "sample_check"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "sample_with_landmarks_final.jpg")
        cv2.imwrite(save_path, frame_vis)
        print(f"Saved sample frame with landmarks to: {save_path}")

    return resized_frames, landmarks_all, motion, velocity

# Save all data
def save_all(video_folder, output_video_folder, output_landmark_folder, output_motion_folder, output_velocity_folder, class_labels):
    os.makedirs(output_video_folder, exist_ok=True)
    os.makedirs(output_landmark_folder, exist_ok=True)
    os.makedirs(output_motion_folder, exist_ok=True)
    os.makedirs(output_velocity_folder, exist_ok=True)

    visualized_once = False

    for class_name in class_labels:
        input_class_folder = os.path.join(video_folder, class_name)
        output_v = os.path.join(output_video_folder, class_name)
        output_l = os.path.join(output_landmark_folder, class_name)
        output_m = os.path.join(output_motion_folder, class_name)
        output_vel = os.path.join(output_velocity_folder, class_name)

        os.makedirs(output_v, exist_ok=True)
        os.makedirs(output_l, exist_ok=True)
        os.makedirs(output_m, exist_ok=True)
        os.makedirs(output_vel, exist_ok=True)

        for video_name in os.listdir(input_class_folder):
            if not video_name.lower().endswith((".mp4", ".avi", ".mov")):
                continue

            video_path = os.path.join(input_class_folder, video_name)
            v_out = os.path.join(output_v, video_name.replace('.mp4', '.npy'))
            l_out = os.path.join(output_l, video_name.replace('.mp4', '_landmarks.npy'))
            m_out = os.path.join(output_m, video_name.replace('.mp4', '_motion.npy'))
            vel_out = os.path.join(output_vel, video_name.replace('.mp4', '_velocity.npy'))

            if os.path.exists(v_out) and os.path.exists(l_out) and os.path.exists(m_out) and os.path.exists(vel_out):
                print(f"Skipping {video_name}, already processed.")
                continue

            frames, landmarks, motions, velocities = extract_video_frames_landmarks_motion(
                video_path, FRAMES, IMG_SIZE,
                visualize=not visualized_once
            )

            if frames is not None:
                np.save(v_out, frames)
                np.save(l_out, landmarks)
                np.save(m_out, motions)
                np.save(vel_out, velocities)
                print(f"Saved {v_out}")
                visualized_once = True
            else:
                print(f"Failed: {video_path}")

# Run Preprocessing
save_all("dataset/Train", "transformed/dataset.npy/Train", "transformed/landmarks.npy/Train", "transformed/motion.npy/Train", "transformed/velocity.npy/Train", CLASSES)
save_all("dataset/Val", "transformed/dataset.npy/Val", "transformed/landmarks.npy/Val", "transformed/motion.npy/Val", "transformed/velocity.npy/Val", CLASSES)
save_all("dataset/Test", "transformed/dataset.npy/Test", "transformed/landmarks.npy/Test", "transformed/motion.npy/Test", "transformed/velocity.npy/Test", CLASSES)

print("Preprocessing Complete!")


Using device: cuda


libEGL warning: DRI3: Screen seems not DRI3 capable
libEGL warning: DRI3: Screen seems not DRI3 capable
MESA: error: ZINK: failed to choose pdev
libEGL warning: egl: failed to create dri2 screen
I0000 00:00:1747551154.490131    1835 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1747551154.530348   45572 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (AMD Radeon(TM) Graphics)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1747551154.698339   45558 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747551154.757319   45557 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747551157.796186   45568 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIO

Saved sample frame with landmarks to: sample_check/sample_with_landmarks_final.jpg
Saved transformed/dataset.npy/Train/Non-suspicious/Video 01-030.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-028.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-035.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-037.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-038.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-039.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-052.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-058.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-062.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-069.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-072.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-078.npy
Saved transformed/dataset.npy/Train/Non-suspicious/Video 02-079.npy
Saved transformed/dataset.npy/Tra

In [ ]:
# Visualize the landmarks in the picture

import numpy as np
import cv2
import os

# Paths
video_npy_path = "dataset_v20.216.npy/Test/Non-suspicious/Video 21-117.npy"
landmark_npy_path = "landmarks_v20_216.npy/Test/Non-suspicious/Video 21-117_landmarks.npy"

# Load once
video_frames = np.load(video_npy_path, mmap_mode='r')  # Use memory mapping (faster!)
landmarks = np.load(landmark_npy_path, mmap_mode='r')  # Same!

# Pick which frame
frame_idx = 4  # 0 to 19

# Just slice 1 frame
frame = video_frames[frame_idx]  # No need to load full into RAM
frame = (frame * 255).astype(np.uint8)  # De-normalize

h, w = frame.shape[:2]

# Slice landmarks for that frame
frame_landmarks = landmarks[frame_idx]
frame_landmarks = (frame_landmarks * np.array([w, h])).astype(np.int32)

# Draw landmarks quickly
for (x, y) in frame_landmarks:
    if x > 0 and y > 0:  # Only draw if valid
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

# Show
cv2.imshow("Frame with Landmarks", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
# ignore this. Haven't refined this one yet

import os
import numpy as np
import torch
import torchvision.io as io
import torchvision.transforms as T
import cv2
import mediapipe as mp

# Constants
IMG_SIZE = 216
FRAMES = 20
CHANNELS = 3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CLASSES = ['Non-suspicious', 'Suspicious']

print(f"Using device: {DEVICE}")

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,    # <--- this helps detect finer details like irises
    min_detection_confidence=0.5
)

# Use first 98 indices
selected_indices = list(range(98))  # <-- simple and clean

# Resize transform
resize_transform = T.Resize((IMG_SIZE, IMG_SIZE))

# Frame and Landmark Extraction
def extract_video_frames_landmarks_motion(video_path, num_frames, img_size, visualize=False):
    try:
        video, _, _ = io.read_video(video_path, pts_unit='sec')
    except Exception as e:
        print(f"Error reading {video_path}: {e}")
        return None, None, None, None

    total_frames = video.shape[0]
    if total_frames < 1:
        return None, None, None, None

    indices = torch.linspace(0, total_frames - 1, num_frames).long()
    frames = video[indices]
    frames = frames.permute(0, 3, 1, 2).float() / 255.0
    frames = frames.cpu()

    resized_frames = []
    landmarks_all = []

    for frame in frames:
        frame_np = frame.permute(1, 2, 0).numpy()
        frame_bgr = (frame_np * 255).astype(np.uint8)
        frame_bgr = cv2.cvtColor(frame_bgr, cv2.COLOR_RGB2BGR)

        results = face_mesh.process(frame_bgr)

        frame_landmarks = []
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            for idx in selected_indices:
                if idx < len(face_landmarks.landmark):
                    lm = face_landmarks.landmark[idx]
                    frame_landmarks.append([lm.x, lm.y])
                else:
                    frame_landmarks.append([0, 0])
        else:
            frame_landmarks = [[0, 0]] * len(selected_indices)

        frame_tensor = torch.tensor(frame_np).permute(2, 0, 1)
        resized = resize_transform(frame_tensor)
        resized = resized.permute(1, 2, 0).numpy()

        resized_frames.append(resized)
        landmarks_all.append(frame_landmarks)

    resized_frames = np.array(resized_frames)  # (T, 216, 216, 3)
    landmarks_all = np.array(landmarks_all)    # (T, 98, 2)

    # Compute Motion (delta)
    motion = landmarks_all[1:] - landmarks_all[:-1]  # (T-1, 98, 2)
    motion = np.vstack([np.zeros((1, 98, 2)), motion])  # pad first frame with zeros

    # Compute Velocity
    time_gap = 0.75
    velocity = motion / time_gap

    # Visualization (optional)
    if visualize:
        frame_idx = 0
        frame_vis = (resized_frames[frame_idx] * 255).astype(np.uint8)
        h, w = frame_vis.shape[:2]
        frame_landmarks = landmarks_all[frame_idx]
        frame_landmarks_scaled = (frame_landmarks * np.array([w, h])).astype(np.int32)

        for (x, y) in frame_landmarks_scaled:
            if x > 0 and y > 0:
                cv2.circle(frame_vis, (x, y), 2, (0, 255, 0), -1)

        save_dir = "sample_check"
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "sample_with_98_landmarks.jpg")
        cv2.imwrite(save_path, frame_vis)
        print(f"Saved sample frame with landmarks to: {save_path}")

    return resized_frames, landmarks_all, motion, velocity


In [1]:
import numpy as np
import pandas as pd
import os

def convert_npy_to_csv(npy_folder, output_folder, file_type="landmarks"):
    os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(npy_folder):
        if not file_name.endswith('.npy'):
            continue
        
        npy_path = os.path.join(npy_folder, file_name)
        data = np.load(npy_path)  # shape: (T, 71, 2)

        if len(data.shape) != 3 or data.shape[1:] != (71, 2):
            print(f"Skipping {file_name} (unexpected shape: {data.shape})")
            continue

        rows = []
        for frame_idx, frame_data in enumerate(data):
            row = {"Frame": frame_idx}
            for point_idx, (x, y) in enumerate(frame_data):
                prefix = file_type.capitalize()  # Capitalizes first letter only
                row[f"{prefix}_{point_idx}_X"] = x
                row[f"{prefix}_{point_idx}_Y"] = y
            rows.append(row)

        df = pd.DataFrame(rows)

        # Save CSV
        csv_name = file_name.replace('.npy', f'_{file_type}.csv')
        csv_path = os.path.join(output_folder, csv_name)
        df.to_csv(csv_path, index=False)
        print(f"Saved: {csv_path}")

# Example usage
convert_npy_to_csv(
    npy_folder="transformed/landmarks.npy/Train/Suspicious",
    output_folder="transformed/landmarks_csv/Train/Suspicious",
    file_type="landmarks"
)

convert_npy_to_csv(
    npy_folder="transformed/motion.npy/Train/Suspicious", 
    output_folder="transformed/motion_csv/Train/Suspicious",
    file_type="motion"
)

convert_npy_to_csv(
    npy_folder="transformed/velocity.npy/Train/Suspicious",  
    output_folder="transformed/velocity_csv/Train/Suspicious",
    file_type="velocity"
)


Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-003_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-007_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-014_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-015_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-021_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-025_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-028_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-029_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-039_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-050_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Suspicious/Video 01-051_landmarks_landmarks.csv
Saved: transformed/landmarks_csv/Train/Susp